# NB_050124T0739_fine_tuning_with_trainer_api

## 1.Goal

fine-tuning model "bert-base-uncased" on  dataset  "glue mrpc" using trainer API in hugging face

# 2. Steps


Using Trainer class we must define:

    - tokenizer
    - datacollector
    - model
    - training_args
    - compute_metrics
    - train_dataset
    - eval_dataset

## 3.Tools

In [1]:
#!pip install datasets evaluate transformers[sentencepiece] install transformers[torch] scipy scikit-learn

     -------------------------------------- 507.1/507.1 KB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 84.1/84.1 KB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 8.2/8.2 MB 2.8 MB/s eta 0:00:00
     -------------------------------------- 365.5/365.5 KB 3.8 MB/s eta 0:00:00
     -------------------------------------- 115.3/115.3 KB 3.4 MB/s eta 0:00:00
     -------------------------------------- 166.4/166.4 KB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 78.3/78.3 KB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 24.6/24.6 MB 2.1 MB/s eta 0:00:00
     -------------------------------------- 133.3/133.3 KB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 15.8/15.8 MB 1.5 MB/s eta 0:00:00
     -------------------------------------- 330.1/330.1 KB 1.9 MB/s eta 0:00:00
     -------------------------------------- 152.8/152.8 KB 1.3 MB/s eta 0:00:00
     -------------------------------------

You should consider upgrading via the 'D:\Projects\toy\skills-HuggingFace\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import torch

In [3]:
torch.cuda.is_available()

True

In [17]:
import scipy
import sklearn

In [4]:
import transformers

d:\Projects\toy\skills-HuggingFace\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 4.Implementation

### 4.0.Initializing raw dataset and checkpoint

In [9]:
from datasets import load_dataset

# Loading the MRPC (Microsoft Research Paraphrase Corpus) dataset from the GLUE benchmark.
raw_datasets = load_dataset("glue", "mrpc")

checkpoint = "bert-base-uncased"  # model name in huggingFace hub

### 4.1.Tokenizer

In [10]:
from transformers import AutoTokenizer

# Loading the tokenizer corresponding to the 'bert-base-uncased' checkpoint.
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

### 4.2. Datacollector

In [11]:
from transformers import DataCollatorWithPadding

# Creating a data collator that will dynamically pad the inputs received, to the maximum length in a batch.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### 4.3. Model

In [7]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 4.4.Training args

In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="test-trainer", evaluation_strategy="epoch"
)

### 4.5.Compute metrics

In [18]:
import evaluate
import numpy as np

metric = evaluate.load("glue", "mrpc")


def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

### 4.6.Train and evalution datasets

In [19]:
# Defining a function to tokenize a pair of sentences.
def tokenize_function(example):
    # Tokenizes a pair of sentences and ensures truncation to the maximum length the model can handle.
    return tokenizer(
        example["sentence1"], example["sentence2"], truncation=True
    )


# Applying the tokenize_function to all examples in the dataset using map, processing in batches for efficiency.
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map: 100%|██████████| 1725/1725 [00:00<00:00, 31090.81 examples/s]


#### EDA

raw_datasets

tokenized_datasets

### 4.7.Fine-tune with Trainer class

In [21]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [22]:
trainer.train()

  0%|          | 0/1377 [00:00<?, ?it/s]You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
                                                  
 34%|███▎      | 464/1377 [00:17<01:35,  9.60it/s]

{'eval_loss': 0.41969034075737, 'eval_accuracy': 0.8284313725490197, 'eval_f1': 0.8841059602649006, 'eval_runtime': 1.3522, 'eval_samples_per_second': 301.723, 'eval_steps_per_second': 37.715, 'epoch': 1.0}


 36%|███▋      | 500/1377 [00:18<00:30, 28.99it/s]

{'loss': 0.5557, 'learning_rate': 3.184458968772695e-05, 'epoch': 1.09}


                                                  
 67%|██████▋   | 924/1377 [00:33<00:45,  9.95it/s]

...

 73%|███████▎  | 1000/1377 [00:36<00:11, 31.47it/s]

{'loss': 0.3358, 'learning_rate': 1.3689179375453886e-05, 'epoch': 2.18}


100%|██████████| 1377/1377 [00:50<00:00, 27.01it/s]

{'eval_loss': 0.5870423913002014, 'eval_accuracy': 0.8578431372549019, 'eval_f1': 0.8993055555555555, 'eval_runtime': 1.3582, 'eval_samples_per_second': 300.405, 'eval_steps_per_second': 37.551, 'epoch': 3.0}
{'train_runtime': 50.9817, 'train_samples_per_second': 215.842, 'train_steps_per_second': 27.01, 'train_loss': 0.37521212154657146, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.37521212154657146, metrics={'train_runtime': 50.9817, 'train_samples_per_second': 215.842, 'train_steps_per_second': 27.01, 'train_loss': 0.37521212154657146, 'epoch': 3.0})